## Setting up Street Fighter

- Importing gym and retro.
- Loading the ROM file.
- Analysing the game space.
- Testing the game.

In [1]:
# Import libraries

import gym, retro

import time  # For slowing down fights

In [3]:
# Optional: Check versions

import os
import sys

print(f"Python version: {sys.version}")
print(f"Retro version: {retro.__version__}")
print(f"Gym version: {gym.__version__}")

Python version: 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
Retro version: 0.8.0
Gym version: 0.21.0


In [4]:
# Optional: Check retro-compatible rom games

# retro.data.list_games()

In [5]:
# Start game environment
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')


# Closes game environment since we can only run one at a time
# env.close()

In [6]:
# Sample the actions available - MultiBinary
env.action_space.sample()

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0], dtype=int8)

In [7]:
# Sample the observation space
env.observation_space.sample()

array([[[ 12,  33,  62],
        [ 67,  31,  35],
        [255, 134,  69],
        ...,
        [166, 107, 227],
        [ 58, 193, 154],
        [143, 233,  16]],

       [[191,  69, 139],
        [189,  28, 193],
        [131, 173, 113],
        ...,
        [ 88, 158, 100],
        [233, 180, 228],
        [218, 187,  57]],

       [[ 72, 250, 122],
        [ 63,  46,  97],
        [208,  65, 171],
        ...,
        [156, 126,  69],
        [ 61, 213, 171],
        [130,  72, 231]],

       ...,

       [[130, 145,  36],
        [ 33,  64,  70],
        [ 74,  98,  69],
        ...,
        [249, 196,  38],
        [166, 178,  89],
        [227,  96, 102]],

       [[ 76, 105, 197],
        [ 48,   3, 117],
        [ 52, 106, 228],
        ...,
        [ 37,  97, 207],
        [230, 225, 223],
        [205,  34, 105]],

       [[ 36,  44, 255],
        [141, 144, 200],
        [ 86, 188,  69],
        ...,
        [208, 133, 206],
        [215,  14, 254],
        [ 38, 206,   2]]

In [8]:
# Test to see everything working

# Reset game to starting state
obs = env.reset()

# Flag to false
done = False

# We only play one game
for game in range(1):

    # If game is not over.
    while not done:
        if done:
            # We reset the game
            obs = env.reset()

        # Render environment
        env.render()

        # We take random actions inside the environment
        obs, reward, done, info = env.step(env.action_space.sample())

        # We slow down the renders so they are watchable
        time.sleep(0)

        # We print the reward
        print(reward)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


KeyboardInterrupt: 

In [ ]:
# Once the testing is finished we close the environment and see what happened.

env.close()
info

NameError: name 'info' is not defined

## Preprocessing the Environment

Observation preprocessing:
1. Calculate change in pixels to capture movement (frame delta).
2. Increase game efficiency by grayscaling and reshaping frames from 200x256x3 to 84x84x1  (153,600 pixels vs 7,056) for faster training.

Action preprocessing:
1. Filtering actions (parameters).
2. Redefine reward functions based on score (possibility of basing it on enemy_health and health).

In [ ]:
# Install open cv
%pip install opencv-python matplotlib

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'opencv-python,'
You should consider upgrading via the 'c:\Code\Street Fighter\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
# Import the libraries necessary for data preprocessing.

from gym import Env  # Base environment class for a wrapper
from gym.spaces import MultiBinary, Box  # Ensure we pick the correct action space type. (Space shapes for the environment)

import numpy as np  # To calculate frame delta
import cv2  # For grayscaling

from matplotlib import pyplot as plt  # For plotting observation images

In [3]:
# Create custom environment
class StreetFighter(Env):
    def __init__(self):

        # Inherit from our base environment
        super().__init__()

        # Specify action and observation spaces
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)  # We create our observation space based on the new size and colors
        self.action_space = MultiBinary(12)  # We replicate the base action environment

        # Startup and instance the game
        # The second parameter will limit actions to only valid ones.
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions = retro.Actions.FILTERED)

    def reset(self):
        # Return first frame, preprocess the frame, and define score back to 0.

        self.previous_frame = np.zeros(self.game.observation_space.shape)

        obs = self.game.reset()  # Will return our observation
        obs = self.preprocess(obs)  # We preprocess the observation

        self.health = 176
        self.enemy_health = 176
        
        # Game delta = Current_frame - Previous_frame
        # Preprocess
        self.previous_frame = obs

        # Attribute to hold delta score.
        self.score = 0

        return obs
    
    def preprocess(self, observation):
        # Grayscale, and resize frame
        
        # Grayscaling
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)

        # Resizing
        resize = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_CUBIC)
        
        channel = np.reshape(resize, (84, 84, 1))  # We add the grayscale layer since its what gym expects

        return channel

    def step(self, action):
        # We take a step, preprocess the observation, calculate frame delta and reshape the reward function

        # Take a step
        obs, reward, done, info = self.game.step(action)  # New step based on an action

        obs = self.preprocess(obs)  # We preprocess the observation

        # Frame delta

        # We subtract the current one from the previous one and then we set the current as the last one.
        frame_delta = obs  # - self.previous_frame
        # self.previous_frame = obs

        # Reshape the reward function based on relative score
        # reward = info['score'] - self.score  # Current reward minus the previous score
        # self.score = info['score']  # We set our score to the current score.
        delta_enemy = self.enemy_health - info['enemy_health']
        delta_self = info['health'] - self.health

        reward = delta_enemy * 2 + delta_self


       # Update values
        self.health = info['health']
        self.enemy_health = info['enemy_health'] 

        return frame_delta, reward, done, info


    def render(self, *args, **kwargs):
        # We render the game
        self.game.render()

    def close(self):
        # We close the game
        self.game.close()

In [ ]:
# We close any environment that could be open
env.close()

In [ ]:
env = StreetFighter()  # We instance the created class

In [ ]:
env.observation_space.shape

(84, 84, 1)

In [ ]:
env.action_space

MultiBinary(12)

In [ ]:
# Test to see everything working

# Reset game to starting state
obs = env.reset()

# Flag to false
done = False

# We only play one game
for game in range(1):

    # If game is not over.
    while not done:
        if done:
            # We reset the game
            obs = env.reset()

        # Render environment
        env.render()

        # We take random actions inside the environment
        obs, reward, done, info = env.step(env.action_space.sample())

        # We slow down the renders so they are watchable
        time.sleep(0)

        # We print the reward
        if reward > 0:
            print(reward)

1000
500
500
1000
500
1500
100
100
100
100
100
100
100
100
100
100
1000
1000
1000
1000
1000
1000
300
300
400
1000
100
1000
100
100


: 

## Hyperparameter tuning

We will use PyTorch, Stable Baselines3 and Optuna to get the model's best training parameters.

For PPO (Proximal Policy Optimization) we will tune the following hyperparameters:
- n_steps: batch size (frames in buffer)
- gamma: discount rate for calculating returns
- learning_rate: learning coefficient for optimizer
- clip_range: clipping amount for advantage calculation
- gae_lambda: advantages smoothing parameter

PyTorch: https://pytorch.org/get-started/locally/

In [ ]:
%pip install torch==2.1.2+cu121 torchvision==0.16.2+cu121 torchaudio==2.1.2+cu121 --extra-index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu118Note: you may need to restart the kernel to use updated packages.



You should consider upgrading via the 'c:\Code\Street Fighter\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Stable Baselines3: https://stable-baselines3.readthedocs.io/en/master/guide/install.html

In [ ]:
%pip install stable-baselines3==1.7.0

You should consider upgrading via the 'c:\Code\Street Fighter\venv\Scripts\python.exe -m pip install --upgrade pip' command.



  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 8.5.0
    Uninstalling importlib-metadata-8.5.0:
      Successfully uninstalled importlib-metadata-8.5.0
  Attempting uninstall: stable-baselines3
    Found existing installation: stable-baselines3 2.4.1
    Uninstalling stable-baselines3-2.4.1:
      Successfully uninstalled stable-baselines3-2.4.1


Optuna: https://optuna.org/#installation

In [ ]:
%pip install optuna

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for PyYAML: filename=PyYAML-6.0.3-cp38-cp38-win_amd64.whl size=45380 sha256=252c5234634e15e50543d1793d271f5c6474ca5cb34a3c84e2f917e6d1062dc8
  Stored in directory: c:\users\sss\appdata\local\pip\cache\wheels\60\9b\bc\8c54c7c157a008210f4d6ddfce9e9c5274162b18c5832f8e24
Successfully built PyYAML
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Code\Street Fighter\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
%pip install "shimmy>=2.0"

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Code\Street Fighter\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
import optuna  # Importing the optimization framework that allows to both train and tune at the same time
import torch
import os  # For exporting the model
from stable_baselines3 import PPO  # PPO algorithm for RL
from stable_baselines3.common.evaluation import evaluate_policy  # Metric calculation of agent performance
from stable_baselines3.common.monitor import Monitor  # SB3 Monitor for logging
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack  # Vec wrappers to vectorize and frame stack

c:\Code\Street Fighter\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Directories where saved optimization models are going to be saved

LOG_DIR = './logs/'  # SB3 has the ability to log out to a support log
OPT_DIR = './opt/'  # Location to save every single model after every try

In [ ]:
# Hyperparameter function to return test hyperparameters - define the objective function

def optimize_ppo(trial):  # i.e. objective
    return {
        # Ranges of possible values that will be optimized
        'n_steps': trial.suggest_int('n_steps', 2048, 8192, step=64),  # SB3 requires  the range to be a multiple of 64
        'gamma': trial.suggest_loguniform('gamma', 0.8, 0.999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range': trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda': trial.suggest_uniform('gae_lambda', 0.8, 0.99),
    }

# When we train we will get a set of best parameters

In [6]:
# Hyperparameter function to run a training loop and return mean 
device = torch.device("cpu")  # Fuerza CPU

def optimize_agent(trial):
    # A try - except section can prevent the model from breaking mid-training

    model_params = optimize_ppo(trial)  # Variable where we store the parameters from the previous function

    # Create environment
    env = StreetFighter()
    env = Monitor(env, LOG_DIR)  # We specify the location where monitor values will be exported to
    env = DummyVecEnv([lambda: env])  # We wrap the environment on a DummyVec
    env = VecFrameStack(env, 4, channels_order='last')  # We will stack 4 different frames

    # Create training algorithm
    # model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)  # We unpack the model parameters obtained from the tuner and pass them to the PPO model
    model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, device=device, **model_params)
    model.learn(total_timesteps=30000)  # We train the model. Longer timesteps means a better model, but also a longer training time. 100k is good, 30k is quick but inaccurate
    
    # Evaluate model
    mean_reward = evaluate_policy(model, env, n_eval_episodes=5)  # We unpack the results obtained from evaluate policy. We will evaluate the model on 5 different games (more == better)
    env.close()

    SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
    model.save(SAVE_PATH)  # We save all models to get the best one

    # We have to give optuna a value it expects, so if its a tuple we return only an int
    if isinstance(mean_reward, (tuple, list)):
        mean_reward = mean_reward[0]

    return mean_reward 
    try:
        pass

    except Exception as e:
        return -1000  # Model did not work, we resume training


In [ ]:
# Tuning

study = optuna.create_study(direction='maximize')  # We create the experiment / study that seeks to maximize the mean reward
study.optimize(optimize_agent, n_trials=10, n_jobs=1)  # We optimize the study based on the agent created, and how many sets we will set. 10 is good for testing, 100+ is recommended for a good model

# NOTE: Using 100k timesteps on the model and 100 trials can take a long time to train (depending on the strength of the gpu from a few hours to a couple of days)

# If we wanted to speed things up whilst keeping accuracy, we could raise n_jobs, however retro does not support more than one environment at once. We can fix
# this by using retrowrapper: https://github.com/MaxStrange/retrowrapper. This allows for multiple instances at once which exponentially speeds trainig up.

[I 2025-09-26 11:52:05,195] A new study created in memory with name: no-name-bcaa0e69-136c-4310-8d92-a314a6d3bf32
C:\Users\SSS\AppData\Local\Temp\ipykernel_8404\2097929532.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial.suggest_loguniform('gamma', 0.8, 0.999),
C:\Users\SSS\AppData\Local\Temp\ipykernel_8404\2097929532.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
C:\Users\SSS\AppData\Local\Temp\ipykernel_8404\2097929532.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/

In [ ]:
study.best_params

{'n_steps': 7360,
 'gamma': 0.8152363058260785,
 'learning_rate': 7.627838127572661e-05,
 'clip_range': 0.3129437660456802,
 'gae_lambda': 0.9822166342841949}

In [ ]:
# To pass it through a model we use
# model = PPO.load(os.path.join(OPT_DIR, 'trial_0_best_model.zip'))

In [ ]:
best_params_video = {
    'n_steps': 7488,
    'gamma': 0.9085173842732223,
    'learning_rate': 5e-7,
    'clip_range': 0.39105070719865653,
    'gae_lambda': 0.8376637411768156
}

## Fine Tuning

In [ ]:
# Import base callback

from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
# Setup Callback

class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
    
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        
        return True

In [7]:
CHECKPOINT_DIR = './train/'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)  # We will save the model every 10k steps

In [ ]:
model_params = study.best_params
model_params['learning_rate'] = 5e-7  # Might improve performance

In [9]:
# Model definition

# env.close()
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, device=device, **model_params)

model.load(os.path.join(OPT_DIR, 'trial_24_best_model.zip'))  # We reload previous weights from HPO

In [ ]:
# Training
model.learn(total_timesteps=100000, callback=callback)  # Bigger is better, for example 5 million
# model.learn(total_timesteps=5000000)

Logging to ./logs/PPO_11
-----------------------------
| time/              |      |
|    fps             | 461  |
|    iterations      | 1    |
|    time_elapsed    | 15   |
|    total_timesteps | 7360 |
-----------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 8.46e+03     |
|    ep_rew_mean          | 1.25e+04     |
| time/                   |              |
|    fps                  | 263          |
|    iterations           | 2            |
|    time_elapsed         | 55           |
|    total_timesteps      | 14720        |
| train/                  |              |
|    approx_kl            | 6.018814e-06 |
|    clip_fraction        | 0            |
|    clip_range           | 0.313        |
|    entropy_loss         | -8.32        |
|    explained_variance   | 2.4e-05      |
|    learning_rate        | 5e-07        |
|    loss                 | 50.5         |
|    n_updates            | 10    

## Model Testing and Evaluating

We can load and visualize the training result.

In [10]:
model = PPO.load('./train/best_model_5460000.zip', device=device)

c:\Code\Street Fighter\venv\lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: an integer is required (got type bytes)
  warnings.warn(
c:\Code\Street Fighter\venv\lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: an integer is required (got type bytes)
  warnings.warn(
c:\Code\Street Fighter\venv\lib\site-packages\stable_baselines3\common\base_class.py:719: UserWarning: You are probably loading a model saved with SB3 < 1.7.0, we deactivated exact_match so you can save the model again to avoid issues in the future (see https://github.com/DLR-RM/stable-baselines3/issues/1233 for more info). Original error: Error(s) in loading state_dict for ActorCriticCnnPolicy:
	Missing key(s) in state_dict: "pi_features_extr

In [11]:
mean_reward, _ = evaluate_policy(model, env, render=False, n_eval_episodes=5)
mean_reward

176.0

In [ ]:
# Test to see everything working

# Reset game to starting state
obs = env.reset()

# Flag to false
done = False

# We only play one game
for game in range(1):

    # If game is not over.
    while not done:
        if done:
            # We reset the game
            obs = env.reset()

        # Render environment
        env.render()

        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)

        # We slow down the renders so they are watchable
        time.sleep(0.01)

        # We print the reward
        if reward > 0:
            print(reward)

[60.]
[18.]
[74.]
[110.]
[70.]
[22.]
[57.]
[62.]
[56.]
[58.]
[14.]
[72.]
[12.]
[74.]
[96.]
[12.]
[52.]
[119.]
[52.]
[52.]
[50.]
[52.]
[12.]
[70.]
[65.]


: 